In [1]:
import codecs
import csv
import json
import os
from os import listdir
import sys
import re
import operator
import random
import time
import spacy

In [9]:
root_dir = ("/Users/patrickmcneillie/data_hub/naccr")
out_dir = ("/Users/patrickmcneillie/data_hub/naccr")

path_to_data_directory = ("/Users/patrickmcneillie/data_hub/naccr/text_file_v3")

nlp = spacy.load('en_core_web_sm')

In [10]:
def open_csv_file_comma(csv_file_path):
    with open(csv_file_path, 'r', encoding='ISO-8859-1') as infile:
        data_reader = csv.reader(infile, delimiter=',', quotechar='"')
        data_list_object = list(data_reader)
    return data_list_object

In [11]:
def save_csv_file_rows(rows, process_file):
    with codecs.open(process_file, encoding='utf-8', mode='w') as fulltext_file:
        writer = csv.writer(fulltext_file, quoting=csv.QUOTE_ALL)
        writer.writerows(rows)
    return

In [12]:
def save_text_file(lines, process_file):
    with codecs.open(process_file, encoding='utf-8', mode='w') as out:
        for row in lines:
            out.write(row + '\n')
    return

In [13]:
def save_text_file_add(lines, process_file):
    with codecs.open(process_file, encoding='utf-8', mode='a+') as out:
        for row in lines:
            out.write(row + '\n')
    return

In [14]:
def string_cleaning(string):
##add padding to other punctuation
    string = re.sub(r'"', '', string)
    string = re.sub(r'(_+)', '_', string)
    string = re.sub(r'(\.+)', '.', string)
    string = re.sub(r',', ' , ', string)
    string = re.sub(r'\\X0D\\\\X0A\\', '\n\r', string)
    string = re.sub(r'(\d)(\s|)(x)(\s|)(\d)', r'\1 \3 \5', string)
    string = re.sub(r"\s{2,}", " ", string)
    
    return string;

In [18]:
def section_labeling(all_headers, inscope_headers, inscope_index, row, x1):
    
    #global new_file_list
    new_file_list = []
    noun_phase_chunk = []
    
    for indx in inscope_index:
        
        focus = row[indx]

        if all_headers[indx] == 'PATIENT_DISPLAY_ID':
            new_file_list.append("Patient_ID :")
            new_file_list.append(focus)
            pt_id = focus
        elif all_headers[indx] == 'SITE':
            new_file_list.append("SITE_ID :")
            new_file_list.append(focus)
            site_id = focus
        else:
            item = all_headers[indx]
            if 'TEXT_PATH_' in item:
                item = re.sub(r'TEXT_PATH_', '', item)
            else:
                pass

            sect_id = ("%s :" % (item))
            new_file_list.append(sect_id)
            focus_new = string_cleaning(focus)
            doc = nlp(focus_new)
            for sent in doc.sents:
                new_file_list.append(sent.text)
                doc2 = nlp(sent.text)
                for chunk in doc2.noun_chunks:
                    noun_phase_chunk.append(chunk.text)
            
    file_id = (site_id + "_" + pt_id + "_" + str(x1))
    new_file_name = ("%s/%s.txt" % (path_to_data_directory, file_id))
    save_text_file(new_file_list, new_file_name)
    noun_phase_file = ("/Users/patrickmcneillie/data_hub/naccr/noun_phase_v1.txt")
    save_text_file_add(noun_phase_chunk, noun_phase_file)
    
    return;

In [19]:
current_file = ('%s/%s' % (root_dir, 'train.csv'))
rows_data = open_csv_file_comma(current_file)


all_headers = rows_data[0]
inscope_headers = ['SITE', 'PATIENT_DISPLAY_ID', 'TEXT_PATH_CLINICAL_HISTORY', 'TEXT_PATH_COMMENTS', 'TEXT_PATH_FORMAL_DX', 'TEXT_PATH_FULL_TEXT', 'TEXT_PATH_GROSS_PATHOLOGY', 'TEXT_PATH_MICROSCOPIC_DESC', 'TEXT_PATH_NATURE_OF_SPECIMENS', 'TEXT_PATH_STAGING_PARAMS', 'TEXT_PATH_SUPP_REPORTS_ADDENDA']
inscope_index = []
for head in inscope_headers:
    indexy = all_headers.index(head)
    inscope_index.append(indexy)


x1 = 1
while x1 < len(rows_data):

    row = rows_data[x1]
    section_labeling(all_headers, inscope_headers, inscope_index, row, x1)
    
    x1 = x1 + 1

print('complete')

complete


In [ ]:
def make_sections():

    line2 = line.lstrip()
    if re.match(r'((history/chief complaint|procedure|operative report|pathology report|vital signs|vs|general|heent|head/neck|eyes|ent|neck|lymph nodes|ln|nodes|cardiovascular|cardiac|chest|heart|respiratory|lungs|dermatology|derm|skin|integument|integ|breast|breasts|breast/chest|abdomen|abd|gastrointestional|gi|genitourinary|gu|rectal|pelvic|rectal/pelvic|musculoskeletal|msk|ms|extremities|estremities|vascular|neurologic|neurological|neuro|psychiatric|psych|endocrine|hematologic|heme|data review|pertinent diagnostic tests|genetics|radiology|radiology review|imaging|pathology|pathology is as follows|addendum|add|attending addendum|attending note |attending note|attending summary|attending summary/addendum|social history|associated medical conditions|routine health maintenance|review of systems|reason for consultation|reason for consult|reason for referral|reason for visit|plan|plan/discussion|physical examination|past surgical history|past medical history|past medical problems|pmh|past medical and surgical history|past medical/surgical history|cancer history|other medical conditions|other surgeries|obstetrical/gynecologic|ob/gyn|obstetric history|past oby-gyn history|past obstetrical/gynecological history|ob|ob history|past gynecologic history|past gyn history|past gyne history|gynecology|gyn history|gyn/menstrual history|gyn|gynecologic history|gynecological history|gynecology history|note|medical problems|impression/plan|impression|imrpession|history of present illness|history|healthcare maintenance|health care maintenance|health care management|health maintenance|chief complaint|chief complaint/identifying data|identifying data|discussion and plan|discussion/plan|discussion|diagnosis|family history|screening history|age appropriate cancer screening|age appropriate screening|cancer screening|medications|home medications|allergies|current medications)(:|\n|\r))', line2, flags= re.I):
        johns = re.match(r'((history/chief complaint|procedure|operative report|pathology report|vital signs|vs|general|heent|head/neck|eyes|ent|neck|lymph nodes|ln|nodes|cardiovascular|cardiac|chest|heart|respiratory|lungs|dermatology|derm|skin|integument|integ|breast|breasts|breast/chest|abdomen|abd|gastrointestional|gi|genitourinary|gu|rectal|pelvic|rectal/pelvic|musculoskeletal|msk|ms|extremities|estremities|vascular|neurologic|neurological|neuro|psychiatric|psych|endocrine|hematologic|heme|data review|pertinent diagnostic tests|genetics|radiology|radiology review|imaging|pathology|pathology is as follows|addendum|add|attending addendum|attending note |attending note|attending summary|attending summary/addendum|social history|associated medical conditions|routine health maintenance|review of systems|reason for consultation|reason for consult|reason for referral|reason for visit|plan|plan/discussion|physical examination|past surgical history|past medical history|past medical problems|pmh|past medical and surgical history|past medical/surgical history|cancer history|other medical conditions|other surgeries|obstetrical/gynecologic|ob/gyn|obstetric history|past oby-gyn history|past obstetrical/gynecological history|ob|ob history|past gynecologic history|past gyn history|past gyne history|gynecology|gyn history|gyn/menstrual history|gyn|gynecologic history|gynecological history|gynecology history|note|medical problems|impression/plan|impression|imrpession|history of present illness|history|healthcare maintenance|health care maintenance|health care management|health maintenance|chief complaint|chief complaint/identifying data|identifying data|discussion and plan|discussion/plan|discussion|diagnosis|family history|screening history|age appropriate cancer screening|age appropriate screening|cancer screening|medications|home medications|allergies|current medications)(:|\n|\r))', line2, flags= re.I)
        jimmy = johns.group(0)

        listy1.append(i)
        listy2.append(jimmy)
        section_set.add(jimmy)
    
    return;

In [ ]:
'Final Diagnosis 1'
'Gross Description 1'
'Sections are submitted as follows'
'1A , 1B:'
'1C through 1F'
'1G , 1H'
'1I , 1J'
'2A.'
'Procedures/Addenda SUPPLEMENTAL REPORT Date Ordered'
'Status'
'Signed Out Date Complete'
'by'
'Date Reported'


In [ ]:
def file_save(lines_final,outfile):
    global my_dict
    with codecs.open(outfile, 'w', encoding='utf-8') as out:
        full_text = ' '.join(lines_final)
        out.write(full_text)

    return;

In [ ]:
def token_dictionary(line):
    global my_dict

    line_d = re.sub(r'(\.)(\s)', ' ', line)
    line_toks = line_d.split()
    
    for word in line_toks:
        if word in my_dict.keys():
            my_dict[word] = my_dict[word] + 1
        else:
            my_dict[word] = 1
    
    return

In [ ]:
dir_list = ['bc_cleaned_1']
output = 'bc_genia'

my_dict = dict()
section_set = set()
first_section = []
last_section = []

for directory in dir_list:
    source_dir = ("%s/%s" % (root_dir, directory))
    process_dir = ("%s/%s" % (out_dir, output))

    docset = listdir(source_dir)
    
    print('Current Directory: %s processing.......' % directory)

    x1 = 0
    while x1 < len(docset):
        doc = docset[x1]
        infile = ("%s/%s" % (source_dir, doc))
        outfile = ("%s/%s" % (process_dir, doc))
        data = codecs.open(infile, 'r', encoding='utf-8')
        stringy = data.readlines()
        data.close()
        
        listy1 = []
        listy2 = []
        final_out = []
        restringy = []

        for i, line in enumerate(stringy):

            line2 = line.lstrip()
            if re.match(r'((history/chief complaint|procedure|operative report|pathology report|vital signs|vs|general|heent|head/neck|eyes|ent|neck|lymph nodes|ln|nodes|cardiovascular|cardiac|chest|heart|respiratory|lungs|dermatology|derm|skin|integument|integ|breast|breasts|breast/chest|abdomen|abd|gastrointestional|gi|genitourinary|gu|rectal|pelvic|rectal/pelvic|musculoskeletal|msk|ms|extremities|estremities|vascular|neurologic|neurological|neuro|psychiatric|psych|endocrine|hematologic|heme|data review|pertinent diagnostic tests|genetics|radiology|radiology review|imaging|pathology|pathology is as follows|addendum|add|attending addendum|attending note |attending note|attending summary|attending summary/addendum|social history|associated medical conditions|routine health maintenance|review of systems|reason for consultation|reason for consult|reason for referral|reason for visit|plan|plan/discussion|physical examination|past surgical history|past medical history|past medical problems|pmh|past medical and surgical history|past medical/surgical history|cancer history|other medical conditions|other surgeries|obstetrical/gynecologic|ob/gyn|obstetric history|past oby-gyn history|past obstetrical/gynecological history|ob|ob history|past gynecologic history|past gyn history|past gyne history|gynecology|gyn history|gyn/menstrual history|gyn|gynecologic history|gynecological history|gynecology history|note|medical problems|impression/plan|impression|imrpession|history of present illness|history|healthcare maintenance|health care maintenance|health care management|health maintenance|chief complaint|chief complaint/identifying data|identifying data|discussion and plan|discussion/plan|discussion|diagnosis|family history|screening history|age appropriate cancer screening|age appropriate screening|cancer screening|medications|home medications|allergies|current medications)(:|\n|\r))', line2, flags= re.I):
                johns = re.match(r'((history/chief complaint|procedure|operative report|pathology report|vital signs|vs|general|heent|head/neck|eyes|ent|neck|lymph nodes|ln|nodes|cardiovascular|cardiac|chest|heart|respiratory|lungs|dermatology|derm|skin|integument|integ|breast|breasts|breast/chest|abdomen|abd|gastrointestional|gi|genitourinary|gu|rectal|pelvic|rectal/pelvic|musculoskeletal|msk|ms|extremities|estremities|vascular|neurologic|neurological|neuro|psychiatric|psych|endocrine|hematologic|heme|data review|pertinent diagnostic tests|genetics|radiology|radiology review|imaging|pathology|pathology is as follows|addendum|add|attending addendum|attending note |attending note|attending summary|attending summary/addendum|social history|associated medical conditions|routine health maintenance|review of systems|reason for consultation|reason for consult|reason for referral|reason for visit|plan|plan/discussion|physical examination|past surgical history|past medical history|past medical problems|pmh|past medical and surgical history|past medical/surgical history|cancer history|other medical conditions|other surgeries|obstetrical/gynecologic|ob/gyn|obstetric history|past oby-gyn history|past obstetrical/gynecological history|ob|ob history|past gynecologic history|past gyn history|past gyne history|gynecology|gyn history|gyn/menstrual history|gyn|gynecologic history|gynecological history|gynecology history|note|medical problems|impression/plan|impression|imrpession|history of present illness|history|healthcare maintenance|health care maintenance|health care management|health maintenance|chief complaint|chief complaint/identifying data|identifying data|discussion and plan|discussion/plan|discussion|diagnosis|family history|screening history|age appropriate cancer screening|age appropriate screening|cancer screening|medications|home medications|allergies|current medications)(:|\n|\r))', line2, flags= re.I)
                jimmy = johns.group(0)

                listy1.append(i)
                listy2.append(jimmy)
                section_set.add(jimmy)

            restringy.append(line)


        for x2 in range(len(listy1)):

            if x2 == 0 and (len(listy1) == 0):
                section0 = restringy
                for liney in section0:
                    liney2 = liney.lstrip()
                    if liney2 != '':
                        liney2 = re.sub(r'\n|\r', ' ', liney2)
                        liney2 = re.sub(r"\s{2,}", " ", liney2)
                        final_out.append(liney2)

            elif x2 == 0 and (len(listy1) == 1):

                section1 = restringy[:listy1[x2]]
                for liney in section1:
                    liney2 = liney.lstrip()
                    if liney2 != '':
                        liney2 = re.sub(r'\n|\r', ' ', liney2)
                        liney2 = re.sub(r"\s{2,}", " ", liney2)
                        final_out.append(liney2)

                section2_title = re.sub(r'\n|\r', '', listy2[x2])
                section2_title_new = ('\n===SECTION: %s===\n' % (section2_title))
                final_out.append(section2_title_new)
                section2 = restringy[listy1[x2]:]
                for liney0 in section2:
                    liney = liney0.lstrip()
                    liney1 = liney.replace(section2_title, '')
                    liney2 = liney1.lstrip()
                    if liney2 != '':
                        liney2 = re.sub(r'\n|\r', ' ', liney2)
                        liney2 = re.sub(r"\s{2,}", " ", liney2)
                        final_out.append(liney2)

            elif x2 == 0 and (len(listy1) > 1):

                section1 = restringy[:listy1[x2]]
                #######DELETE##########
                document_name = (doc + '\n')
                first_section.append(document_name)
                firsty = []
                #######DELETE##########
                for liney in section1:
                    liney2 = liney.lstrip()
                    if liney2 != '':
                        liney2 = re.sub(r'\n|\r', ' ', liney2)
                        liney2 = re.sub(r"\s{2,}", " ", liney2)
                        ########final_out.append(liney2)
                        firsty.append(liney2)
                
                first_out = ' '.join(firsty)
                first_out2 = re.sub(r"\s{2,}", " ", first_out)
                first_section.append(first_out2 + '\n')

                section2_title = re.sub(r'\n|\r', '', listy2[x2])
                section2_title_new = ('\n===SECTION: %s===\n' % (section2_title))
                final_out.append(section2_title_new)
                section2 = restringy[listy1[x2]:listy1[x2+1]]
                for liney0 in section2:
                    liney = liney0.lstrip()
                    liney1 = liney.replace(section2_title, '')
                    liney2 = liney1.lstrip()
                    if liney2 != '':
                        liney2 = re.sub(r'\n|\r', ' ', liney2)
                        liney2 = re.sub(r"\s{2,}", " ", liney2)
                        final_out.append(liney2)


            elif x2 == (len(listy1)-1):

                section_final_title = re.sub(r'\n|\r', '', listy2[x2])
                section_final_title_new = ('\n===SECTION: %s===\n' % (section_final_title))
                final_out.append(section_final_title_new)
                section_final = restringy[listy1[x2]:]
                #######DELETE##########
                document_name = (doc + '\n')
                last_section.append(document_name)
                lasty = []
                #######DELETE##########
                for liney0 in section_final:
                    liney = liney0.lstrip()
                    liney1 = liney.replace(section_final_title, '')
                    liney2 = liney1.lstrip()
                    if liney2 != '':
                        liney2 = re.sub(r'\n|\r', ' ', liney2)
                        liney2 = re.sub(r"\s{2,}", " ", liney2)
                        #########final_out.append(liney2)
                        lasty.append(liney2)
                
                last_out = ' '.join(lasty)
                last_out2 = re.sub(r"\s{2,}", " ", last_out)
                really_last = []               
                if re.search(r'(DICT:)(.*)', last_out2, flags= re.I):
                    s = re.search(r'(DICT:)(.*)', last_out2, flags= re.I)
                    last_section.append(s.group(0) + '\n')
                    liney3 = last_out2.replace(s.group(0), '')
                    final_out.append(liney3)
                    
                elif re.search(r'(INTERNAL CC:)|(Fax Request Sent to Server for:)|(Report Electronically Signed Out by)|(EXTERNAL CC:)|(\(\*\*\[MD Initials\]\) cc:)', last_out2, flags= re.I):                   
                    if re.search(r'(INTERNAL CC:)', last_out2, flags= re.I):
                        s1 = re.search(r'(INTERNAL CC:)(.*)', last_out2, flags= re.I)
                        really_last.append(s1.group(0))
                        
                    if re.search(r'(Fax Request Sent to Server for:)', last_out2, flags= re.I):
                        s2 = re.search(r'(Fax Request Sent to Server for:)(.*)', last_out2, flags= re.I)
                        really_last.append(s2.group(0))
                        
                    if re.search(r'(Report Electronically Signed Out by)', last_out2, flags= re.I):
                        s3 = re.search(r'(Report Electronically Signed Out by)(.*)', last_out2, flags= re.I)
                        really_last.append(s3.group(0))
                        
                    if re.search(r'(EXTERNAL CC:)', last_out2, flags= re.I):
                        s4 = re.search(r'(EXTERNAL CC:)(.*)', last_out2, flags= re.I)
                        really_last.append(s4.group(0))
                        
                    if re.search(r'(\(\*\*\[MD Initials\]\) cc:)', last_out2, flags= re.I):
                        s5 = re.search(r'(\(\*\*\[MD Initials\]\) cc:)(.*)', last_out2, flags= re.I)
                        really_last.append(s5.group(0))
                        
                    really_last.sort(key=len, reverse=True)
                    removey = really_last[0]
                    last_section.append(removey + '\n')
                    liney3 = last_out2.replace(removey, '')
                    final_out.append(liney3)
                    
                else:
                    final_out.append(last_out2)
                    

            else:

                section_next_title = re.sub(r'\n|\r', '', listy2[x2])
                section_next_title_new = ('\n===SECTION: %s===\n' % (section_next_title))
                final_out.append(section_next_title_new)
                section_next = restringy[listy1[x2]:listy1[x2+1]]
                #######DELETE##########
                document_name = (doc + '\n')
                last_section.append(document_name)
                lasty = []
                #######DELETE##########
                for liney0 in section_next:
                    liney = liney0.lstrip()
                    liney1 = liney.replace(section_next_title, '')
                    liney2 = liney1.lstrip()
                    if liney2 != '':
                        liney2 = re.sub(r'\n|\r', ' ', liney2)
                        liney2 = re.sub(r"\s{2,}", " ", liney2)
                        ###########final_out.append(liney2)
                        lasty.append(liney2)
                        
                last_out = ' '.join(lasty)
                last_out2 = re.sub(r"\s{2,}", " ", last_out)
                if re.search(r'(DICT:)(.*)', last_out2, flags= re.I):
                    s = re.search(r'(DICT:)(.*)', last_out2, flags= re.I)
                    last_section.append(s.group(0) + '\n')
                    liney3 = last_out2.replace(s.group(0), '')
                    final_out.append(liney3)
                    #line = line.replace(s.group(0), '')
                else:
                    final_out.append(last_out2)
                    


        file_save(final_out,outfile)
        #for future_line in final_out:
            #if re.search(r'===SECTION:', future_line):
                #pass
            #else:
                #token_dictionary(future_line)
                
        #document_name = (doc + '\n')
        #first_section.append(doc)
        #first_section.append(final_out[0])
        
        #for last_line in final_out:
            #if re.search(r'(DICT:)(.*)', last_line, flags= re.I):
                #s = re.search(r'(DICT:)(.*)', last_line, flags= re.I)
                #last_section.append(s.group(0))
                #line = line.replace(s.group(0), '')
        
        #print(final_out)
        x1 = x1 + 1

print('cases_processed')

john2 = sorted(my_dict.items(), key=lambda x: x[1], reverse=True)
with open(dictionary_file, encoding='utf-8', mode='w') as dict_file:
    writer = csv.writer(dict_file)
    writer.writerows(john2)

print('dictionary_completed')

In [ ]:
dir_list = ['wo_meta_data']
output = 'sect_sent_list'

my_dict = dict()

for directory in dir_list:
    source_dir = ("%s/%s" % (root_dir, directory))
    process_dir = ("%s/%s" % (out_dir, output))

    docset = listdir(source_dir)
    
    print('Current Directory: %s processing.......' % directory)

    x1 = 0
    while x1 < len(docset):
        doc = docset[x1]
        infile = ("%s/%s" % (source_dir, doc))
        outfile = ("%s/%s" % (process_dir, doc))
        data = codecs.open(infile, 'r', encoding='utf-8')
        stringy = data.readlines()
        data.close()
        
        final_out = []
        for line in (stringy):
            if re.search(r'===SECTION:', line):
                line = line.lstrip()
                section = ('%s' % line)
                final_out.append(section)
            else:
                line = re.sub(r'(\s)([0-9]\))(\s*.*?)', '\n\t\\2', line)
                line = re.sub(r'([a-z]\.)(\s*)([A-Z])', '\\1 |.| \\3', line)
                line = re.sub(r'([A-Z]\.)(\s*)([A-Z])', '\\1 |.| \\3', line)
                line = re.sub(r'([0-9]\.)(\s*)([A-Z])', '\\1 |.| \\3', line)
                line = re.sub(r'([0-9]\.)(\s)([0-9])', '\\1 |.| \\3', line)
                line = re.sub(r'(\)\.)(\s*)([A-Z])', '\\1 |.| \\3', line)
                line = re.sub(r'([a-z]\.)(\s*)([0-9])', '\\1 |.| \\3', line)
                line = re.sub(r'(%\.)(\s*)([0-9])', '\\1 |.| \\3', line)
                line2 = line.split('|.|')
                for item in line2:
                    item = re.sub(r'\r|\n|\t', '', item)
                    item = item.lstrip()
                    if len(item.strip()) != 0:
                        #print('NEW: %d::%s' % (len(item.strip()),item.strip()))
                        final_out.append('\t' + item.strip() + '\n')
                        token_dictionary(item.strip())

        file_save(final_out,outfile)
        #print(final_out)
        x1 = x1 + 1

print('cases_processed')

john2 = sorted(my_dict.items(), key=lambda x: x[1], reverse=True)
with open(dictionary_file, encoding='utf-8', mode='w') as dict_file:
    writer = csv.writer(dict_file)
    writer.writerows(john2)

print('dictionary_completed')

In [ ]:

'''
TEXT_PATH_CLINICAL_HISTORY
TEXT_PATH_COMMENTS
TEXT_PATH_FORMAL_DX
TEXT_PATH_FULL_TEXT
TEXT_PATH_GROSS_PATHOLOGY
TEXT_PATH_MICROSCOPIC_DESC
TEXT_PATH_NATURE_OF_SPECIMENS
TEXT_PATH_STAGING_PARAMS
TEXT_PATH_SUPP_REPORTS_ADDENDA
'''


In [ ]:
row = rows_data[0]

add_id1 = row[0]
stie_id1 = row[2]
pt_id1 = row[4]
clin_hx1 = row[7]
comment1 = row[8]
form_dx1 = row[9]
full_text1 = row[10]
path1 = row[11]
micro1 = row[12]
spec1 = row[13]
staging1 = row[14]
adden1 = row[15]

x1 = 1
while x1 < len(rows_data):
    
    new_file_list = []
    
    row = rows_data[x1]
    
    add_id = row[0]
    stie_id = row[2]
    pt_id = row[4]
    clin_hx = row[7]
    comment = row[8]
    form_dx = row[9]
    full_text = row[10]
    path = row[11]
    micro = row[12]
    spec = row[13]
    staging = row[14]
    adden = row[15]
        
    Patient_id = ("Patient_ID : %s" % pt_id)
    new_file_list.append([Patient_id])
    Label_id = ("Label_ID : %s" % stie_id)
    new_file_list.append([Label_id])
    
    #new_file_list.append([add_id1])
    #new_file_list.append([add_id])
    #new_file_list.append([stie_id1])
    #new_file_list.append([stie_id])
    #new_file_list.append([pt_id1])
    #new_file_list.append([pt_id])
    new_file_list.append([clin_hx1])
    new_file_list.append([clin_hx])
    new_file_list.append([comment1])
    new_file_list.append([comment])
    new_file_list.append([form_dx1])
    new_file_list.append([form_dx])
    new_file_list.append([full_text1])
    new_file_list.append([full_text])
    new_file_list.append([path1])
    new_file_list.append([path])
    new_file_list.append([micro1])
    new_file_list.append([micro])
    new_file_list.append([spec1])
    new_file_list.append([spec])
    new_file_list.append([staging1])
    new_file_list.append([staging])
    new_file_list.append([adden1])
    new_file_list.append([adden])
    
    
    file_id = (pt_id + "_" + add_id)
    path_to_data_directory = ("/Users/patrickmcneillie/data_hub/naccr/text_files")
    new_file_name = ("%s/%s.txt" % (path_to_data_directory, file_id))
    
    save_text_file(new_file_list, new_file_name)
    

    x1 = x1 + 1

print('complete')

In [ ]:
inscope_headers_clean = []
for item in inscope_headers:
    if 'TEXT_PATH_' in item:
        item = re.sub(r'TEXT_PATH_', '', item)
    else:
        pass
    
    inscope_headers_clean.append(item)

row = rows_data[0]

add_id1 = row[0]
stie_id1 = row[2]
pt_id1 = row[4]
clin_hx1 = row[7]
comment1 = row[8]
form_dx1 = row[9]
full_text1 = row[10]
path1 = row[11]
micro1 = row[12]
spec1 = row[13]
staging1 = row[14]
adden1 = row[15]

x1 = 1
while x1 < len(rows_data):
    
    new_file_list = []
    
    row = rows_data[x1]
    
    add_id = row[0]
    stie_id = row[2]
    pt_id = row[4]
    clin_hx = row[7]
    comment = row[8]
    form_dx = row[9]
    full_text = row[10]
    path = row[11]
    micro = row[12]
    spec = row[13]
    staging = row[14]
    adden = row[15]
        
    Patient_id = ("Patient_ID : %s" % pt_id)
    new_file_list.append(Patient_id)
    Label_id = ("Label_ID : %s" % stie_id)
    new_file_list.append(Label_id)
    
    #new_file_list.append([add_id1])
    #new_file_list.append([add_id])
    #new_file_list.append([stie_id1])
    #new_file_list.append([stie_id])
    #new_file_list.append([pt_id1])
    #new_file_list.append([pt_id])
    new_file_list.append(clin_hx1)
    new_file_list.append(clin_hx)
    new_file_list.append(comment1)
    new_file_list.append(comment)
    new_file_list.append(form_dx1)
    new_file_list.append(form_dx)
    new_file_list.append(full_text1)
    new_file_list.append(full_text)
    new_file_list.append(path1)
    new_file_list.append(path)
    new_file_list.append(micro1)
    new_file_list.append(micro)
    new_file_list.append(spec1)
    new_file_list.append(spec)
    new_file_list.append(staging1)
    new_file_list.append(staging)
    new_file_list.append(adden1)
    new_file_list.append(adden)
    
    
    file_id = (pt_id + "_" + add_id)
    path_to_data_directory = ("/Users/patrickmcneillie/data_hub/naccr/text_files_v2")
    new_file_name = ("%s/%s.txt" % (path_to_data_directory, file_id))
    
    save_text_file(new_file_list, new_file_name)
    

    x1 = x1 + 1

print('complete')